In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 200)
plt.style.use('ggplot')

In [3]:
df = pd.read_csv("hf://datasets/maharshipandya/spotify-tracks-dataset/dataset.csv")
df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


# understanding the data

In [4]:
df.shape

(114000, 21)

In [5]:
df.columns

Index(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

In [6]:
df.dtypes

Unnamed: 0            int64
track_id             object
artists              object
album_name           object
track_name           object
popularity            int64
duration_ms           int64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
track_genre          object
dtype: object

# data preparation

## dropping irrelevant columns

In [7]:
df.rename(columns={'Unnamed: 0': 'index_col'}, inplace=True)
df=df.set_index('index_col')

In [8]:
df=df.drop(columns=['artists','track_id','album_name','track_name'])
df

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
index_col,,,,,,,,,,,,,,,,
0,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music


In [9]:
df.reset_index(drop=True)

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,21,384999,False,0.172,0.2350,5,-16.393,1,0.0422,0.6400,0.928000,0.0863,0.0339,125.995,5,world-music
113996,22,385000,False,0.174,0.1170,0,-18.318,0,0.0401,0.9940,0.976000,0.1050,0.0350,85.239,4,world-music
113997,22,271466,False,0.629,0.3290,0,-10.895,0,0.0420,0.8670,0.000000,0.0839,0.7430,132.378,4,world-music
113998,41,283893,False,0.587,0.5060,7,-10.889,1,0.0297,0.3810,0.000000,0.2700,0.4130,135.960,4,world-music


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114000 entries, 0 to 113999
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   popularity        114000 non-null  int64  
 1   duration_ms       114000 non-null  int64  
 2   explicit          114000 non-null  bool   
 3   danceability      114000 non-null  float64
 4   energy            114000 non-null  float64
 5   key               114000 non-null  int64  
 6   loudness          114000 non-null  float64
 7   mode              114000 non-null  int64  
 8   speechiness       114000 non-null  float64
 9   acousticness      114000 non-null  float64
 10  instrumentalness  114000 non-null  float64
 11  liveness          114000 non-null  float64
 12  valence           114000 non-null  float64
 13  tempo             114000 non-null  float64
 14  time_signature    114000 non-null  int64  
 15  track_genre       114000 non-null  object 
dtypes: bool(1), float64(9), i

## missing values

In [11]:
df.isna().sum()

popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

there are no missing values

# scaling and normalization

## scaling

In [12]:
# for min_max scaling
from sklearn.preprocessing import MinMaxScaler

In [13]:
# Columns to scale
cols_to_scale=['duration_ms','loudness','liveness','tempo','time_signature']
# Min-Max Scaling
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

df.head()

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
index_col,,,,,,,,,,,,,,,,
0,73,0.044043,False,0.676,0.4610,1,0.791392,0,0.1430,0.0322,0.000001,0.3580,0.715,0.361245,0.8,acoustic
1,55,0.028566,False,0.420,0.1660,1,0.597377,1,0.0763,0.9240,0.000006,0.1010,0.267,0.318397,0.8,acoustic
2,57,0.040255,False,0.438,0.3590,0,0.736123,1,0.0557,0.2100,0.000000,0.1170,0.120,0.313643,0.8,acoustic
3,71,0.038557,False,0.266,0.0596,0,0.573701,1,0.0363,0.9050,0.000071,0.1320,0.143,0.746758,0.6,acoustic
4,82,0.037969,False,0.618,0.4430,2,0.737103,1,0.0526,0.4690,0.000000,0.0829,0.167,0.492863,0.8,acoustic


# encoding

## encoding boolean data

In [14]:
df['explicit'] = df['explicit'].astype(int)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114000 entries, 0 to 113999
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   popularity        114000 non-null  int64  
 1   duration_ms       114000 non-null  float64
 2   explicit          114000 non-null  int64  
 3   danceability      114000 non-null  float64
 4   energy            114000 non-null  float64
 5   key               114000 non-null  int64  
 6   loudness          114000 non-null  float64
 7   mode              114000 non-null  int64  
 8   speechiness       114000 non-null  float64
 9   acousticness      114000 non-null  float64
 10  instrumentalness  114000 non-null  float64
 11  liveness          114000 non-null  float64
 12  valence           114000 non-null  float64
 13  tempo             114000 non-null  float64
 14  time_signature    114000 non-null  float64
 15  track_genre       114000 non-null  object 
dtypes: float64(11), int64(4),

# data preprocessing

## train-test split

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


# Separate target from predictors
y = df.popularity
X = df.drop(['popularity'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
#my_cols = encoded_cat_cols + numerical_cols
X_train = X_train_full[numerical_cols].copy()
X_valid = X_valid_full[numerical_cols].copy()

In [17]:
X_train.head()

,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
index_col,,,,,,,,,,,,,,
57684,0.053393,0,0.785,0.700,9,0.808705,0,0.0406,0.017900,0.002510,0.0823,0.2960,0.501418,0.8
41923,0.047769,0,0.529,0.885,2,0.816584,1,0.1250,0.000027,0.613000,0.1450,0.2990,0.460180,0.8
110236,0.040188,0,0.572,0.956,10,0.849490,0,0.0418,0.010400,0.000188,0.2050,0.0375,0.517718,0.8
76577,0.042159,0,0.195,0.198,1,0.592327,0,0.0360,0.892000,0.000000,0.0777,0.2390,0.347197,0.8
18163,0.032943,1,0.422,0.916,1,0.794296,1,0.8810,0.869000,0.000000,0.9060,0.4970,0.326139,0.8


# modelling

In [18]:
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from lightgbm import LGBMRegressor


In [19]:
reg_models = {
    'random_forest':RandomForestRegressor(),
    'xgboost':GradientBoostingRegressor(),
    'linear_reg':LinearRegression(),
    'ridge':Ridge(),
    'lasso':Lasso(),
    'svr':SVR(),
    'lightgbm':LGBMRegressor()
}

We'll use dictionaries to loop over the models and also store the results in a dictionary

In [20]:
pred_dict = {
    'reg_model' :[],
    'mae_score' :[]
}

In [21]:

for name, model in reg_models.items():
    model.fit(X_train, y_train)
    # Preprocessing of validation data, get predictions
    preds = model.predict(X_valid)
    #validating the model
    mae = mean_absolute_error(y_valid, preds)
    pred_dict['reg_model'].append(name)
    pred_dict['mae_score'].append(mae)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018774 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2571
[LightGBM] [Info] Number of data points in the train set: 91200, number of used features: 14
[LightGBM] [Info] Start training from score 33.190000


In [22]:
pred_dict

{'reg_model': ['random_forest',
  'xgboost',
  'linear_reg',
  'ridge',
  'lasso',
  'svr',
  'lightgbm'],
 'mae_score': [10.701771922745426,
  17.500393926468536,
  18.388226090755794,
  18.388375471151956,
  18.893644736842106,
  18.21254445614759,
  16.49142960138796]}

# loading

In [23]:
# Create a DataFrame
load_df = pd.DataFrame(pred_dict)

In [24]:
load_df

,reg_model,mae_score
0,random_forest,10.701772
1,xgboost,17.500394
2,linear_reg,18.388226
3,ridge,18.388375
4,lasso,18.893645
5,svr,18.212544
6,lightgbm,16.491430


Random forests appear to have the lowest MAE thus making it the best performing model.